In [1]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

gpt-3.5-turbo: ~$0.0015 per 1,000 tokens (input) and ~$0.002 per 1,000 tokens (output).


gpt-4: ~$0.03 per 1,000 tokens (input) and ~$0.06 per 1,000 tokens (output).

In [2]:
model_gpt = "gpt-3.5-turbo"
api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model=model_gpt, 
    temperature=0.4, # Increase creativity
    max_tokens=2000, # Allow for longer responses
    frequency_penalty=0.5, # Reduce repetition
    presence_penalty=0.6, # Encourage new topics 
    api_key=api_key
)
print("GPT Model Initialized")

GPT Model Initialized


In [3]:
from langchain.prompts import PromptTemplate

In [4]:
prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],
     template=(
        "You are a professional financial advisor. Your job is to provide clear, actionable advice "
        "based on the client's financial situation and goals. Use available tools and data sources "
        "to ensure accurate answers.\n\n"
        "Client Question: {client_question}\n\n"
        "Agent's Scratchpad:\n{agent_scratchpad}"
    )
)

In [5]:
from langchain import hub

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from langchain.document_loaders import PyPDFLoader

In [8]:
from langchain.agents import AgentExecutor
from IPython.display import display, Markdown

In [9]:
folder_dir="C:/Users/igara/prediction_proyect_st/books"
all_documents=[]
# docs = loader.load()
for file in os.listdir(folder_dir):
    print(file)
    file_path=os.path.join(folder_dir,file)
    print(file_path)
    loader=PyPDFLoader(file_path)
    docs = loader.load()
        # Split the document into chunks
    documents = RecursiveCharacterTextSplitter(
    chunk_size=10000, 
    chunk_overlap=0
    ).split_documents(docs)
    all_documents.extend(documents)

    print(f"Loaded and split {len(all_documents)} document chunks.")

A-Random-Walk-Down-Wall-Street.pdf
C:/Users/igara/prediction_proyect_st/books\A-Random-Walk-Down-Wall-Street.pdf
Loaded and split 566 document chunks.
Common-Stocks-and-Uncommon-Profits-Philip-Fisher.pdf
C:/Users/igara/prediction_proyect_st/books\Common-Stocks-and-Uncommon-Profits-Philip-Fisher.pdf
Loaded and split 878 document chunks.
Economics-in-One-Lesson_2.pdf
C:/Users/igara/prediction_proyect_st/books\Economics-in-One-Lesson_2.pdf
Loaded and split 1081 document chunks.
Personal Financial Planning Guide.pdf
C:/Users/igara/prediction_proyect_st/books\Personal Financial Planning Guide.pdf
Loaded and split 1635 document chunks.
principles-for-navigating-big-debt-crises-by-ray-dalio.pdf
C:/Users/igara/prediction_proyect_st/books\principles-for-navigating-big-debt-crises-by-ray-dalio.pdf
Loaded and split 2091 document chunks.
The Millionaire Next Door (Thomas J. Stanley, William D. Danko).pdf
C:/Users/igara/prediction_proyect_st/books\The Millionaire Next Door (Thomas J. Stanley, Willi

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [11]:
vector = FAISS.from_documents(all_documents, OpenAIEmbeddings(api_key=api_key))
retriever = vector.as_retriever()
print("Vector store created and retriever initialized.")

Vector store created and retriever initialized.


In [12]:
from langchain_community.tools import DuckDuckGoSearchRun

In [13]:
search = DuckDuckGoSearchRun()

In [14]:
from langchain.tools.retriever import create_retriever_tool

In [15]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for relevant information from the documents. Use this tool for any document-related questions of finance.",
)

In [16]:
tools = [search, retriever_tool]

In [17]:
from langchain.agents import create_tool_calling_agent

In [18]:
agent = create_tool_calling_agent(
    llm=llm, 
    tools=tools, 
    prompt=prompt)

In [19]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [20]:
# query = "What are the key phases of a debt crisis, as described by Ray Dalio?"
# response = agent_executor.invoke({"client_question": query})
# print(query, response)